<a href="https://colab.research.google.com/github/Kiichiro-T/python/blob/main/regression_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [ ]:
dataset = datasets.load_boston()
# print(dataset.DESCR)
x = dataset.data
y = dataset.target

In [ ]:
from sklearn.model_selection import train_test_split

# 訓練用データセット（70%）とテスト用データセット（30%）に分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
N = len(x_train)
ones = np.ones((N,1))
X = np.concatenate((ones, x_train), axis=1)

# 逆行列を計算
xx = np.dot(X.T, X)
xx_inv = np.linalg.inv(xx)
# xt = np.dot(X.T, y_train)
# beta1 = np.dot(xx_inv, xt)
# print('逆行列を計算した場合：')
# print(beta1)

# 連立一次方程式を解く
beta_hat = np.linalg.solve(X.T.dot(X), X.T.dot(y_train))
print('連立一次方程式を解いた場合の　')
print(beta_hat)

# 当てはめ値（回帰値）
y_hat = X.dot(beta_hat)
# print(y_hat)

# 残差
e = y_train - y_hat
# print(e)

# 残差の平方和
Se = e.T.dot(e)
# print(Se)

# 全変動
y_mean = y_train.mean()
a = y_train - y_mean * np.ones(N)
St = a.T.dot(a)
# print(St)

# 自由度
n = x_train.shape[0]
k = x_train.shape[1]
df = n - k

# 決定係数
R2 = 1 - Se / St
print('決定係数:', R2)

# 自由度調整済み決定係数
R2_f = 1 -(Se/df)/(St/n-1)
print('自由度調整済み決定係数:', R2_f)

# 分散共分散行列
s2 = Se / df
V_hat = Se * xx_inv
# print(V_hat)

# 標準誤差
se = np.sqrt(s2)

# テスト用データセットを用いた評価
N_test = len(x_test)
X_test = np.concatenate((np.ones((N_test,1)), x_text), axis=1)
y_test_hat = X_test.dot(beta_hat)
e_test = y_test - y_test_hat
Se_test = e_test.T.dot(e_test)
y_test_mean = y_test.mean()
b = y_test - y_test_mean * np.ones(N_test)
St_test = b.T.dot(b)
n_test = x_test.shape[0]
k_test = x_test.shape[1]
df_test = n_test - k_test
R2_test = 1 - Se_test / St_test
print('テストデータに対する決定係数:', R2_test)
R2_f_test = 1 -(Se_test/df_test)/(St_test/n_test-1)
print('テストデータに対する自由度調整済み決定係数:', R2_f_test)

if R2 > R2_test:
  print('R2>R2_testより性能が悪い（過学習）')

連立一次方程式を解いた場合の　
[ 3.79371077e+01 -1.21310401e-01  4.44664254e-02  1.13416945e-02
  2.51124642e+00 -1.62312529e+01  3.85906801e+00 -9.98516565e-03
 -1.50026956e+00  2.42143466e-01 -1.10716124e-02 -1.01775264e+00
  6.81446545e-03 -4.86738066e-01]
決定係数: 0.7645451026942549
自由度調整済み決定係数: 0.7526507406770253
テストデータに対する決定係数: 0.6733825506400206
テストデータに対する自由度調整済み決定係数: 0.6384939888287393
R2>R2_testより性能が悪い（過学習）


## メソッド化

In [ ]:
def score(beta_hat, x, y):
  N = len(x)
  X = np.concatenate((np.ones((N,1)), x), axis=1)

  # 当てはめ値（回帰値）
  y_hat = X.dot(beta_hat)

  # 残差
  e = y - y_hat

  # 残差の平方和
  Se = e.T.dot(e)

  # 全変動
  y_mean = y.mean()
  a = y - y_mean * np.ones(N)
  St = a.T.dot(a)

  # 自由度
  n = x.shape[0]
  k = x.shape[1]
  df = n - k

  # 決定係数
  R2 = 1 - Se / St
  print('テストデータに対する決定係数:', R2)

  # 自由度調整済み決定係数
  R2_f = 1 -(Se/df)/(St/n-1)
  print('テストデータに対する自由度調整済み決定係数:', R2_f)
  
def calc(x, y):
  N = len(x) # データセット数

  X = np.concatenate((np.ones((N,1)), x), axis=1)

  # 回帰係数
  beta_hat = np.linalg.solve(X.T.dot(X), X.T.dot(y))
  print('回帰係数：')
  print(beta_hat)

  return beta_hat

In [ ]:
beta_hat = calc(x_train, y_train)
score(beta_hat, x_train, y_train)

score(beta_hat, x_test, y_test)

回帰係数：
[ 3.79371077e+01 -1.21310401e-01  4.44664254e-02  1.13416945e-02
  2.51124642e+00 -1.62312529e+01  3.85906801e+00 -9.98516565e-03
 -1.50026956e+00  2.42143466e-01 -1.10716124e-02 -1.01775264e+00
  6.81446545e-03 -4.86738066e-01]
テストデータに対する決定係数: 0.7645451026942549
テストデータに対する自由度調整済み決定係数: 0.7526507406770253
テストデータに対する決定係数: 0.6733825506400317
テストデータに対する自由度調整済み決定係数: 0.6384939888287515


## 標準化したデータセットによる重回帰分析

In [ ]:
x_train_i_mean = x_train.mean(axis=0)
x_train_i_var = x_train.var(axis=0)
x_train_std = (x_train - x_train_i_mean) / np.sqrt(x_train_i_var)
# print('_____________________')
# print(x_train_std)

x_test_i_mean = x_test.mean(axis=0)
x_test_i_var = x_test.var(axis=0)
x_test_std = (x_test - x_test_i_mean) / np.sqrt(x_test_i_var)
# print('_____________________')
# print(x_test_std)

# 確認用
# from sklearn.preprocessing import StandardScaler

# scaler1 = StandardScaler()
# scaler1.fit(x_train)
# a = scaler1.transform(x_train)
# print('_____________________')
# print(a)

# scaler2 = StandardScaler()
# scaler2.fit(x_test)
# b = scaler2.transform(x_test)
# print('_____________________')
# print(b)

beta_hat2 = calc(x_train_std, y_train)
score(beta_hat2, x_train_std, y_train)

score(beta_hat2, x_test_std, y_test)

回帰係数：
[22.74548023 -1.01190059  1.05028027  0.07920966  0.618962   -1.87369102
  2.70526979 -0.27957264 -3.09766486  2.09689998 -1.88606339 -2.26110466
  0.58264309 -3.44049838]
テストデータに対する決定係数: 0.7645451026942549
テストデータに対する自由度調整済み決定係数: 0.7526507406770253
テストデータに対する決定係数: 0.6591396211105117
テストデータに対する自由度調整済み決定係数: 0.6227296607081945


In [ ]:
import scipy as sp

# データセットを読み込み
from sklearn.datasets import load_iris
iris_dataset = load_iris()

x = iris_dataset.data
y = iris_dataset.target
pd.DataFrame(iris_dataset.data, columns=iris_dataset.feature_names)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
# アヤメデータをラベルと入力データに分離する
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

In [ ]:
# from sklearn.linear_model import LinearRegression

# reg_model = LinearRegression()
# reg_model.fit(x_train, y_train)
# print(reg_model.coef_)
# print(reg_model.intercept_)

beta_hat = calc(x_train, y_train)
print(beta_hat)

# beta3（sepal width）の値が小さいのでt検定を行う。
N = len(x_train)
X = np.concatenate((np.ones((N,1)), x_train), axis=1)
y_hat = X.dot(beta_hat)
e = y_train - y_hat
Se = e.T.dot(e)
n = x.shape[0]
k = x.shape[1]
df = n - k
s2 = Se / df

# 標準誤差←間違い、分散共分散行列の対角成分から取得する。
se = np.sqrt(s2)

# t検定
print('H_0: beta3 = 0')
print('H_1: beta3 ≠ 0')
print('α = 0.05')
t = (beta_hat[1] - 0) / se
t_a = sp.stats.t.ppf(0.975, df)
if abs(t) > t_a:
  print('帰無仮説を棄却する')
else:
  print('帰無仮説を棄却しない')

print('sepal widthには統計的に優位な増加や減少を認められない')

回帰係数：
[ 0.23870394 -0.16058723  0.00104505  0.26916947  0.57091235]
[ 0.23870394 -0.16058723  0.00104505  0.26916947  0.57091235]
H_0: beta3 = 0
H_1: beta3 ≠ 0
α = 0.05
帰無仮説を棄却しない


In [ ]:
# 検定は一旦おいておく（2変数？）
# 信頼区間
# 最尤推定